In [1]:
import json

import pandas as pd
from datetime import datetime, timedelta

import trane
import featuretools as ft
import numpy as np

/Users/leix/miniconda3/envs/py36/lib/python3.6/site-packages/featuretools/__init__.py:60: FutureWarning: The next non-bugfix release of Featuretools will not support Python 3.6
  FutureWarning


In [2]:
df = pd.read_csv("data/bike-sampled.csv")
df['date'] = df['date'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))
df = df.sort_values(by=['date'])
df.tail()


,date,hour,usertype,gender,tripduration,temperature,from_station_id,dpcapacity_start,to_station_id,dpcapacity_end
3304,2017-01-31,10,Subscriber,Female,6.966667,37.9,247,19.0,247,19.0
3305,2017-01-31,10,Subscriber,Male,6.483333,37.9,425,15.0,426,19.0
3306,2017-01-31,10,Subscriber,Female,8.250000,37.9,175,19.0,45,15.0
3299,2017-01-31,10,Subscriber,Male,16.266667,37.9,202,15.0,317,23.0
0,2017-01-31,23,Subscriber,Male,3.316667,35.1,230,19.0,131,15.0


In [3]:
entity_col = "__fake_root_entity__"
meta = trane.TableMeta(json.loads(open('data/meta.json').read()))
df, meta = trane.overall_prediction_helper(df, meta)

# MAP str to int
df_ft = df.copy()
str_col_list = ['usertype', 'gender']
str_mappers = {}
for str_col in str_col_list:
    str_to_id = {}
    id_to_str = []
    n_entity = 0
    
    for item in set(df_ft[str_col]):
        str_to_id[item] = n_entity
        id_to_str.append(item)
        n_entity += 1

    if str_col == entity_col:
        df[str_col] = df[str_col].apply(lambda x: str_to_id[x])
    df_ft[str_col] = df_ft[str_col].apply(lambda x: str_to_id[x])
    str_mappers[str_col] = (str_to_id, id_to_str)

    

df_ft.tail()

,date,hour,usertype,gender,tripduration,temperature,from_station_id,dpcapacity_start,to_station_id,dpcapacity_end,__fake_root_entity__
3304,2017-01-31,10,0,0,6.966667,37.9,247,19.0,247,19.0,0
3305,2017-01-31,10,0,1,6.483333,37.9,425,15.0,426,19.0,0
3306,2017-01-31,10,0,0,8.250000,37.9,175,19.0,45,15.0,0
3299,2017-01-31,10,0,1,16.266667,37.9,202,15.0,317,23.0,0
0,2017-01-31,23,0,1,3.316667,35.1,230,19.0,131,15.0,0


In [6]:
cutoff_base = datetime.strptime("2017-01-06", "%Y-%m-%d")
cutoff_end = datetime.strptime("2017-01-31", "%Y-%m-%d")
cutoff_strategy = trane.FixWindowCutoffStrategy(entity_col, cutoff_base, cutoff_end, 1)

features = trane.FeaturetoolsWrapper(df_ft, entity_col, 'date', 
                                        {'hour': ft.variable_types.Categorical,
                                         'usertype': ft.variable_types.Categorical,
                                         'gender': ft.variable_types.Categorical,
                                         'from_station_id': ft.variable_types.Categorical,
                                         'to_station_id': ft.variable_types.Categorical,
                                        }, 'bikes')
features.compute_features(df_ft, cutoff_strategy, 5)

Built 43 features
Elapsed: 00:00 | Remaining: ? | Progress:   0%|          | Calculated: 0/3 chunks2018-10-18 11:54:33,691 featuretools.entityset - WARNING    Using training_window but last_time_index is not set on entity bikes
2018-10-18 11:54:33,986 featuretools.entityset - WARNING    Using training_window but last_time_index is not set on entity bikes
2018-10-18 11:54:34,272 featuretools.entityset - WARNING    Using training_window but last_time_index is not set on entity bikes
2018-10-18 11:54:34,536 featuretools.entityset - WARNING    Using training_window but last_time_index is not set on entity bikes
2018-10-18 11:54:34,793 featuretools.entityset - WARNING    Using training_window but last_time_index is not set on entity bikes
2018-10-18 11:54:35,049 featuretools.entityset - WARNING    Using training_window but last_time_index is not set on entity bikes
2018-10-18 11:54:35,320 featuretools.entityset - WARNING    Using training_window but last_time_index is not set on entity bike

In [10]:
problem_generator = trane.PredictionProblemGenerator(
    table_meta=meta, entity_col=entity_col, time_col="date")

problems = problem_generator.generate()


Success/Attempt = 79/360

Success/Attempt = 271/1394

Success/Attempt = 418/2091


In [11]:
evaluator = trane.PredictionProblemEvaluator(df, 
                                             entity_col=entity_col, 
                                             cutoff_strategy=cutoff_strategy, 
                                             min_train_set=5,
                                             min_test_set=5,
                                             previous_k_as_feature=2,
                                             latest_k_as_test=8
                                             )


In [12]:
result = trane.multi_process_evaluation(evaluator, problems, features)
with open("prob_with_acc.json", "w") as f:
    json.dump(result, f)


100%|██████████| 418/418 [01:11<00:00,  5.88it/s]
